In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

options = Options()

prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
options.add_experimental_option('prefs',prefs)
options.binary_location = '/usr/bin/chromium-browser'
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument("--headless")            #不開啟實體瀏覽器背景執行


base_url = "https://mis.twse.com.tw/stock/etf_nav.jsp?ex=tse"

browser=webdriver.Chrome(options=options)
browser.get(base_url)

time.sleep(3)
html=browser.page_source
soup=BeautifulSoup(html,'html.parser')
div=soup.select_one("div#content")
table=pd.read_html(str(div))
frames = [table[0], table[1], table[3], table[5]]
result=pd.concat(frames,ignore_index=True)
print(result)

In [12]:
result.head()

,Stock Code/Name of Fund,Total Outstanding Units(Annotation 1),Net Changes in Outstanding Units,Market Price,Estimated Net Asset Value(Annotation 2),The Percentage of Estimated Premium/Discount(Annotation 3),Net Asset Value per unit of the Previous Business Day(Annotation 4),ETF issuers' web site,Last update
0,0050 / TTT,871000000,6000000,137.45,136.95,0.37%,142.58,Query NAV,2021/01/22-13:45:00
1,0051 / TMT,10000000,0,45.39,45.49,-0.22%,44.60,Query NAV,2021/01/22-13:45:00
2,0052 / FB TECHNOLOGY,45000000,500000,135.90,132.01,2.95%,135.35,Query NAV,2021/01/22-16:59:20
3,0053 / PTE,5488000,0,67.50,66.96,0.81%,67.96,Query NAV,2021/01/22-13:45:00
4,0054 / PCP,7624000,0,30.54,30.77,-0.75%,30.42,Query NAV,2021/01/22-13:45:00


In [24]:
df = result[['Stock Code/Name of Fund','Market Price','Estimated Net Asset Value(Annotation 2)','The Percentage of Estimated Premium/Discount(Annotation 3)']]
df.columns = ['名稱','市價','淨值','溢價幅度']

In [70]:
# 溢價大於0.5%
discount = df[df["溢價幅度"] > "0.5%" ].sort_values(by="溢價幅度", ascending=False).head(10)
discount

,名稱,市價,淨值,溢價幅度
30,00881 / CATHAY TAIWAN 5G...,18.50,17.94,3.12%
99,00715L / JKO BRENT 2X,5.15,5.00,3.00%
2,0052 / FB TECHNOLOGY,135.90,132.01,2.95%
79,00677U / FB VIX,3.24,2.88,12.50%
27,00742 / SHIN KONG DOMEST...,20.50,20.14,1.76%
13,00663L / TAIEX LEV 2X,96.10,94.47,1.73%
23,00728 / FSITC INDUSTRY E...,27.32,26.98,1.26%
3,0053 / PTE,67.50,66.96,0.81%
10,006208 / FB TW50,79.20,78.60,0.76%
48,00653L / FB NIFTY L2X1,34.97,34.72,0.72%


In [71]:
# 折價小於-3%
premium = df[df["溢價幅度"] < "-3%" ].sort_values("溢價幅度", ascending=False).head(10)
premium

,名稱,市價,淨值,溢價幅度
60,00743 / CATHAY FTSE CHIN...,31.28,32.12,-2.62%
90,00703 / TAISHIN MSCI CHI...,28.04,28.75,-2.47%
31,0061 / PWC300,25.31,25.95,-2.47%
40,00639 / SZSE 100,18.36,18.82,-2.44%
43,00643 / CSZSME,20.01,20.51,-2.44%
89,00702 / CATHAY S&P 500 L...,17.72,18.15,-2.37%
113,00851 / TAISHIN GLOBAL A...,29.35,30.05,-2.33%
64,00783 / FUBON CHINA 500 ...,22.53,23.05,-2.26%
59,00739 / YT MSCI CHINA A,29.89,30.55,-2.16%
47,00652 / FB NIFTY,24.90,25.45,-2.16%
